# CNN Model

In [ ]:
class CNN(nn.Module):

    def __init__(self, act, pooling):
        super(CNN, self).__init__()
        self.act = act
        self.pooling = pooling
        
        self.conv1 = nn.Conv2d(in_channels = 3,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 64,
                               out_channels = 256,
                               kernel_size = 5,
                               stride = 1,
                               padding = 2)
        
        if self.act == 'relu':
            self.act = nn.ReLU()
        elif self.act == 'tanh':
            self.act == nn.Tanh()
        elif self.act == 'sigmoid':
            self.act = nn.Sigmoid()
        else:
            raise ValueError('no valid activation function selected!')
       
        if self.pooling == 'max_pooling': 
            self.pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)
        elif self.pooling == 'avg_pooling':
            self.pooling = nn.AvgPool2d(kernel_size = 2, stride=2)
        else: 
            raise ValueError('Not a valid pooling code')
        
        self.fc = nn.Linear(65536, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.act(x)
        x = self.pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Experiment Code

In [ ]:
def experiment(partition, args):
    
    net = CNN(act = args.act,
              pooling = args.pooling)
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(net, partition, args)    
    
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

# Experiment

In [ ]:
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_basic_CNN_model_max_pooling"

# ====== Model ====== #
args.act = 'relu' 
args.pooling = 'max_pooling'  #'max_pooling' #'avg_pooling'

# ====== Regularization ======= #
args.l2 = 0.00001

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 10

args.train_batch_size = 512
args.test_batch_size = 1024
                
setting, result = experiment(partition, deepcopy(args))
save_exp_result(setting, result)

#### Epoch 0, Acc(train/val): 37.74/44.97, Loss(train/val) 1.77/1.57. Took 34.83 sec
#### Epoch 1, Acc(train/val): 48.48/48.72, Loss(train/val) 1.46/1.43. Took 35.95 sec
#### Epoch 2, Acc(train/val): 53.24/52.47, Loss(train/val) 1.33/1.34. Took 35.36 sec
#### Epoch 3, Acc(train/val): 55.73/54.11, Loss(train/val) 1.26/1.29. Took 35.38 sec
#### Epoch 4, Acc(train/val): 57.77/55.85, Loss(train/val) 1.21/1.25. Took 35.57 sec
#### Epoch 5, Acc(train/val): 59.55/57.13, Loss(train/val) 1.16/1.22. Took 35.40 sec
#### Epoch 6, Acc(train/val): 60.95/58.56, Loss(train/val) 1.12/1.18. Took 35.45 sec
#### Epoch 7, Acc(train/val): 62.58/58.83, Loss(train/val) 1.08/1.18. Took 35.46 sec
#### Epoch 8, Acc(train/val): 63.80/59.77, Loss(train/val) 1.04/1.15. Took 35.43 sec
#### Epoch 9, Acc(train/val): 64.76/60.74, Loss(train/val) 1.02/1.12. Took 35.46 sec